<a href="https://colab.research.google.com/github/simon-bouchard/Book_Recommendation_KNN_with_FastAPI/blob/master/model_notebooks/book_recommendation_EmbeddingNN_for_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
import torch
from torch.nn import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from fastai.collab import *
from fastai.tabular.all import *


In [5]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-03-03 21:49:31--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2025-03-03 21:49:31 (182 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [6]:
# import csv data into dataframes
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [7]:
user_ratings_count = ratings['user'].value_counts()

valid_users = user_ratings_count[user_ratings_count >= 100].index

movie_ratings_count = ratings['isbn'].value_counts()

valid_movies = movie_ratings_count[movie_ratings_count >= 100].index

ratings = ratings[(ratings['user'].isin(valid_users)) & (ratings['isbn'].isin(valid_movies))]
books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [66]:
data = ratings.merge(books, on='isbn').drop(columns=['author'])

dls = CollabDataLoaders.from_df(data, item_name='title', bs=32)
dls.show_batch()

,user,title,rating
0,52199,G Is for Gumshoe (Kinsey Millhone Mysteries (Paperback)),10.0
1,27140,The Perfect Storm : A True Story of Men Against the Sea,0.0
2,256402,Stranger in a Strange Land (Remembering Tomorrow),0.0
3,85701,Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1),0.0
4,182085,Hot Six : A Stephanie Plum Novel (A Stephanie Plum Novel),8.0
5,55892,Guilty as Sin,0.0
6,230522,Ashes to Ashes,0.0
7,16795,"All the Pretty Horses (The Border Trilogy, Vol 1)",0.0
8,55892,"Angus, Thongs and Full-Frontal Snogging: Confessions of Georgia Nicolson",8.0
9,60277,Wicked: The Life and Times of the Wicked Witch of the West,0.0


In [9]:
embs = get_emb_sz(dls)
embs

[(1770, 105), (674, 61)]

In [10]:
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

In [11]:
class DotProductBias(Module):
    def __init__(self, n_users, n_books, n_factors, y_range=(0, 10.5)):
        super().__init__()

        self.user_factors = nn.Embedding(n_users, n_factors)
        self.user_bias = nn.Embedding(n_users, 1)
        self.book_factors = nn.Embedding(n_books, n_factors)
        self.book_bias = nn.Embedding(n_books, 1)
        self.y_range = y_range

    def forward(self, x):
        users = self.user_factors(x[:, 0])
        books = self.book_factors(x[:, 1])
        res = (users * books).sum(dim=1)

        res += self.user_bias(x[:, 0]).squeeze() + self.book_bias(x[:, 1]).squeeze()

        return sigmoid_range(res, *self.y_range)

In [12]:
n_users = len(dls.classes['user'])
n_books = len(dls.classes['title'])

In [56]:
model = DotProductBias(n_users, n_books, 50)
learn = Learner(dls, model, loss_func=lambda pred, target: SmoothL1Loss()(pred, target.squeeze(-1)))
learn.fit_one_cycle(15, 5e-3, wd=0.3)

epoch,train_loss,valid_loss,time
0,4.510223,4.593723,00:04
1,3.715125,4.053117,00:04
2,2.685378,3.121204,00:04
3,2.165043,2.468521,00:04
4,1.793283,2.137282,00:04
5,1.774118,2.077319,00:04
6,1.805164,2.060058,00:04
7,1.724890,2.026021,00:04
8,1.761878,2.029921,00:04
9,1.708505,2.014253,00:04


In [57]:
book_bias = learn.model.book_bias.weight.squeeze()
idxs = book_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Harry Potter and the Goblet of Fire (Book 4)',
 'Harry Potter and the Prisoner of Azkaban (Book 3)',
 "Harry Potter and the Sorcerer's Stone (Book 1)",
 'Tears of the Giraffe (No.1 Ladies Detective Agency)',
 'The Tao of Pooh']

In [64]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,10.5), n_act=100, dropout=0.3):
        super().__init__()
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(n_act, n_act // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(n_act // 2, n_act // 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(n_act // 4, 1))
        self.y_range = y_range

    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

In [ ]:
#model = CollabNN((n_users, 100), (n_books, 100))
model = CollabNN(*embs)
learn = Learner(dls, model, loss_func=SmoothL1Loss())
#learn.lr_find()
learn.fit_one_cycle(20, 1e-3, wd=0.4)

epoch,train_loss,valid_loss,time
0,2.028757,1.979522,00:10
1,1.732312,1.878112,00:10
2,1.723954,1.898382,00:10
3,1.887765,1.924330,00:11
4,1.834054,1.888281,00:10
5,1.903820,1.892030,00:10
6,1.835271,1.893998,00:10
7,1.829553,1.899502,00:10
8,1.789207,1.879464,00:11
9,1.756084,1.881890,00:10


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  book_isbn = df_books.loc[df_books['title'] == book, 'isbn']
  book_isbn = book_isbn.values[0]

  isbn_to_index = {isbn: idx for idx, isbn in enumerate(user_book_matrix.index)}
  query_index = isbn_to_index[book_isbn]

  distances, indices = knn.kneighbors(sparse_matrix[query_index], n_neighbors=5)

  recommended_books = [book, []]

  reversed_indices = indices[0][1:][::-1]  # Skip itself and reverse
  reversed_distances = distances[0][1:][::-1]

  for idx, dist in zip(reversed_indices, reversed_distances):  # Skip itself
    neighbor_isbn = user_book_matrix.index[idx]
    neighbor_book = df_books.loc[df_books['isbn'] == neighbor_isbn, 'title']
    neighbor_book = neighbor_book.values[0]

    recommended_books[1].append([neighbor_book, float(f'{dist:.2f}')])

  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77], ['The Surgeon', 0.77], ['I Know This Much Is True', 0.76], ['The Lovely Bones: A Novel', 0.72]]]
You passed the challenge! 🎉🎉🎉🎉🎉
